In [1]:
import numpy as np

### Labeled bifurcating tree

When we add species to the tree, the number of way of doing that is equal to the number of the branch. The tree of 2 species have 3 branches and every time we add species, we create 2 new branches. Then we can generate $3$ different trees of $3$ species, the $3$-species tree have $5$ branches, so for each of the $3$ different trees we have, we can generate $5$ trees with $4$ species. That gives us $3 \times 5$ trees of $4$-species. So we can generalize that the number of bifurcating tree for $n$-species tree is
	\begin{equation}
	3 \times 5 \times 7 \times 9 \times \dots \times (2n-3)
	\label{1.2}
	\end{equation}

By arranging this, we have,

\begin{align*}
3 \times 5 \times 7 \times 9 \times \dots \times (2n-3)&=\frac{1 \times 2 \times 3 \times 4 \times 5 \times 6 \times 7 \times  \dots \times (2n-4) \times (2n-3)}{2 \times 4 \times 6 \times \dots \times (2n-4)}\\
&=\frac{(2n-3)!}{2^{n-2}(1 \times 2 \times \dots \times (n-2))}
\end{align*}
	
Therefore, the number of birfurcating labeled trees with $n$ species is,

\begin{equation}
\frac{(2n-3)!}{2^{n-2}(n-2)!}
\label{1.3}
\end{equation} 

<img src="root_bif.png" width=300 height=300 />

In [2]:
def possible_tree(tree,species):
    '''
    Input: a tree and a species to add on the tree
    Output: all possible tree when we add the species to each branch of the tree
    '''
    stock=[]
    if type(tree[0])!=list and type(tree[1])==list:
        stock.append([[tree[0],species],tree[1]])
        stock.append([[tree[0],tree[1]],species])
        for i in possible_tree(tree[1],species):
            stock.append([tree[0],i])
    elif type(tree[1])!=list and type(tree[0])==list:
        stock.append([tree[0],[tree[1],species]])
        stock.append([[tree[0],tree[1]],species])
        for i in possible_tree(tree[0],species):
            stock.append([i,tree[1]])
    elif type(tree[0])!=list and type(tree[1])!=list:
        stock.append([[tree[0],species],tree[1]])
        stock.append([tree[0],[tree[1],species]])
        stock.append([[tree[0],tree[1]],species])
    else:
        stock.append([tree,species])
        for i in possible_tree(tree[0],species):
            stock.append([i,tree[1]])
        for j in possible_tree(tree[1],species):
            stock.append([tree[0],j])     
    return stock     

In [3]:
#all structure of tree when we insert species 6
L=possible_tree([[[1, 2], [3, 4]], 5],6)
L

[[[[1, 2], [3, 4]], [5, 6]],
 [[[[1, 2], [3, 4]], 5], 6],
 [[[[1, 2], [3, 4]], 6], 5],
 [[[[1, 6], 2], [3, 4]], 5],
 [[[1, [2, 6]], [3, 4]], 5],
 [[[[1, 2], 6], [3, 4]], 5],
 [[[1, 2], [[3, 6], 4]], 5],
 [[[1, 2], [3, [4, 6]]], 5],
 [[[1, 2], [[3, 4], 6]], 5]]

In [4]:
def labeled(list_of_leaf):
    '''
    Input: a list of the label of leaf, we enumerate is as an integeger
    Output: a list of all possible labelled bifurcating tree for a given different leaf
    '''
    all_tree=[[list_of_leaf[0],list_of_leaf[1]]]
    for i in list_of_leaf[2:]:
        ls=[]
        for L in all_tree:
            trees=possible_tree(L,i)
            for elt in trees:
                ls.append(elt)
            all_tree=ls.copy()
    return all_tree

In [5]:
#all possible labeled tree for 5 species
Q1=labeled([1,2,3,4,5])
len(Q1)

105

###  Most parsimonious trees using Sankoff algorithm

#### Small parsimony problem using Sankoff algorithm

In [6]:
import numpy as np

In [7]:
def vector_cost(S_l, S_r,M):
    '''
    Input: 
            - S_l and S_r: vector cost for two children of a node
            - M: cost matrix
    Output:
            - S_m: vector cost for the node
    '''
    S_m=np.zeros(len(S_r))
    l=np.zeros(len(S_r))
    r=np.zeros(len(S_r))
    for i in range(len(S_l)):
        for j in range(len(S_r)):
            l[j]=S_l[j]+M[i][j]
            r[j]=S_r[j]+M[i][j]
        m_l= min(l)
        m_r=min(r)
        S_m[i]=m_l+m_r
    return S_m

In [8]:
#we have here the cost matrix M
M=[[0,2.5,1,2.5],[2.5,0,2.5,1],[1,2.5,0,2.5],[2.5,1,2.5,0]]

In [9]:
#try to compute the vector cost for one node
vector_cost([2.5,2.5,3.5,3.5], [3.5,3.5,3.5,4.5] ,M)

array([6., 6., 7., 8.])

In [10]:
def Sankoff(tree,s,C,M):
    '''
    This function compute the number of change for one character
    Input:
        - tree: tree
        - s: set of character state
        - C: list of character for all species
        - M: cost matrix
    Output:
        - score,vector cost of the root
    '''
    score=0
    if type(tree[0])!=list and type(tree[1])==list:
        score,root_cost=Sankoff(tree[1],s,C,M)
        l=C[tree[0]-1]
        n1 = [float('inf')] * len(s)
        index=s.index(l)
        n1[index]=0
        root_cost= vector_cost(n1,root_cost,M)
    elif type(tree[1])!=list and type(tree[0])==list:
        score,root_cost=Sankoff(tree[0],s,C,M)
        r=C[tree[1]-1]
        n2 = [float('inf')] * len(s)
        index=s.index(r)
        n2[index]=0
        root_cost= vector_cost(root_cost,n2,M)
    elif type(tree[0])==list and type(tree[1])==list:
        score,root_cost_l=Sankoff(tree[0],s,C,M)
        score,root_cost_r=Sankoff(tree[1],s,C,M)
        root_cost= vector_cost(root_cost_l,root_cost_r,M)
    else:
        l=C[tree[0]-1]
        n1 = [float('inf')] * len(s)
        index=s.index(l)
        n1[index]=0
        r=C[tree[1]-1]
        n2 = [float('inf')] * len(s)
        index=s.index(r)
        n2[index]=0
        root_cost= vector_cost(n1,n2,M)
    score=min(root_cost)
    return score, root_cost   

We use our code here to see if it give the same result as we see in the figure bellow
<img src="Sankoff.png" width=400 height=400 />

In [11]:
#we have a tree here
tree=[[1,2],[3,[4,5]]]
#this is a list of the state of character
s=['A','C','G','T']
#this is the list of observed character in the leaf
C=['C','A','C','A','G']

In [12]:
#compute the parsimony score for the tree above
score,vec=Sankoff(tree,s,C,M)
print('The parsimony score for this phylogeny is',score,'. \n The vector cost in the root of this is',vec)

The parsimony score for this phylogeny is 6.0 . 
 The vector cost in the root of this is [6. 6. 7. 8.]


In [13]:
def Sankoff_tree(tree,data,s,M):
    '''
    This function count the number of change for all the character of species
    Input:
            - Tree
            - data: Matrice which represent all the Character of each species, species in the row and character in the column    
            - s: set of character state
            - M: cost matrix
    Output:
            - count: parsimony score for the given tree
    '''
    count=0
    rw,col=np.shape(data)
    for i in range(col):
        r1=[row[i] for row in data]
        score,root_cost=Sankoff(tree,s,r1,M)
        count+=score
    return count

#### Large parsimony problem with Sankoff algorithm

In [14]:
def parsimonious_Sank(tree_gen,data,s,M):
    '''
    Input: 
            - tree_gen: all possible tree for a given set of species
            - data: Matrice which represent all the Character of each species, species in the row and character in the column    
            - s: set of character state
            - M: cost matrix
    '''
    minim=Sankoff_tree(tree_gen[0],data,s,M) 
    most_parsimonious=[tree_gen[0]]
    for tree in tree_gen[1:]:
        count=Sankoff_tree(tree,data,s,M)
        if count<minim:
            most_parsimonious=[tree]
            minim=count
        elif count==minim:
            most_parsimonious.append(tree)
    return most_parsimonious, minim

## Test of effectiveness of our program

This is a sequence of genome of 4 species, the length of this sequence is 200.

<img src="data.png" width=400 height=400 />


We see on the figure bellow the true tree usign Juckes Cantor Model for this data.

<img src="jc.png" width=200 height=200 />

###  Test of our program using Fitch algorithm for the data bellow

In [15]:
#The sequence of species A
A='CATTGGGGTCATCTACCGTCTTAGCGCAGGGGGGTGGACGTGCATCCTTTGACCGCTATGACCGGGTAATAGTTGCAAAGGAGGTAACATCATTCCCCGGAGCTATAGTAGGGCAATATACCTTTGATCAGACCTTAACCTTGCACCCTCCTGAGCCAGGGAGTCGGGAACTGATTCATCCACAGTCTGGCAGTTTGAGA'
a=[i for i in A]

print('The number of A in the sequence is',a.count('A'),'\n The number of C in the sequence is',a.count('C'),'\n The number of G in the sequence is',a.count('G'),'\nThe number of T in the sequence is',a.count('T'))

The number of A in the sequence is 47 
 The number of C in the sequence is 49 
 The number of G in the sequence is 55 
The number of T in the sequence is 49


In [16]:
##The sequence of species B
B='CATTGGGGTAATCTACCGTCTTAGCGTTGTGAGGTGGGCGTGCATACTTAGACACCTATGACCGGGTAATAGTTGCAAAGGAGGTAACATCATTCCACGGAGGCATAGTCGGGCAATATACCTTTCATCAGACCTTAAGTTTGCACCTTCCTGAGCCATGGAATCGGGAGGTGATTCATCAACAGTCTGCCAGTTTGAGC'
b=[i for i in B]
print('The number of A in the sequence is',b.count('A'),'\n The number of C in the sequence is',b.count('C'),'\n The number of G in the sequence is',b.count('G'),'\nThe number of T in the sequence is',b.count('T'))

The number of A in the sequence is 50 
 The number of C in the sequence is 44 
 The number of G in the sequence is 53 
The number of T in the sequence is 53


In [17]:
C='CATTGGGGTCGTCAACCGCTTTCGCTAAGGGGGGTGGGAGTGCATACTTTGACCCCGATGACCGGCTAATAGTTGCAAAGGATGTAACGTTATTCCACGGATGTATTGTCGGGCACAATACCTGGCCTCAAACCTTGAGTTTGCACCCTCCTGAGCCATGGAATCGGGAGTTGATTCATCAACAGCCGGGCATTTTGAGA'
c=[i for i in C]
print('The number of A in the sequence is',c.count('A'),'\n The number of C in the sequence is',c.count('C'),'\n The number of G in the sequence is',c.count('G'),'\nThe number of T in the sequence is',c.count('T'))

The number of A in the sequence is 46 
 The number of C in the sequence is 47 
 The number of G in the sequence is 56 
The number of T in the sequence is 51


In [18]:
D='CATTGGGGTCATCTACGGGCTTAGCTCAGGGGAGTGGGCGTGCATACTTAGACCCCTATGTCCGGGTAATAGTTGCAAAGGAGGTATCATTATTCCACGGAGGTATTGTCGGGCAAAATACCTTTGATGTGACCTTAAGATTGCACCCTCCTGAGCCTTGGAATCGGGGGCTGGTTCATCAACAGTCCGTCAGTCTGAGA'
d=[i for i in D]
print('The number of A in the sequence is',d.count('A'),'\n The number of C in the sequence is',d.count('C'),'\n The number of G in the sequence is',d.count('G'),'\nThe number of T in the sequence is',d.count('T'))

The number of A in the sequence is 45 
 The number of C in the sequence is 44 
 The number of G in the sequence is 58 
The number of T in the sequence is 53


In [19]:
#create the data set
data=[a,b,c,d]
#generate all possible tree for 4 species (we enumerate species A=1, species B=2, species C=3, species D=4)
tree_gen=labeled([1,2,3,4])

In [20]:
#find the most parsimonious tree
cost_matrix=[[0,1,1,1],[1,0,1,1],[1,1,0,1],[1,1,1,0]]
parsimonious_Sank(tree_gen,data,s,cost_matrix)

([[[1, [3, 4]], 2],
  [1, [2, [3, 4]]],
  [[1, 2], [3, 4]],
  [[[1, 2], 3], 4],
  [[[1, 2], 4], 3]],
 67.0)

These tree is represented in the figure bellow
 <img src="tree1.png" width=100 height=100 /> <img src="tree2.png" width=100 height=100 />
    <img src="tree3.png" width=100 height=100 />
      <img src="tree4.png" width=100 height=100 />
        <img src="tree5.png" width=100 height=100 />

 When we remove the root of the tree above, we have this unrooted tree below 
  <img src="unrootpas.png" width=200 height=200 />

### Genome tree generator from Professor Martin 

In [21]:
import random
import math as M

def jukesCantor(genome,gen_time):
  """
  Input: genome, a character string consisting of a, c, g, and t
         gen_time, genetic time
  Output: a randomly modified string according to the Juke's cantor model
    The following coupled ODEs are integrated from t=0 to t=gen_time

    dot a = -a + (c + g + t) /3
    dot c = -a + (g + t + a) /3
    dot g = -g + (t + a + c) /3
    dot t = -t + (a + c + g) /3
  
    using the solution 
 
    a(t)=(3/4)*exp(-t)*(a(0)-(c(0)+g(0)+t(0))/3)
        +(1/4)*(a(0)+c(0)+g(0)+t(0))

    and similarly for c(t), g(t), and t(t).

    In other words, if 'a' is the initial base, its persistence
    probability after a time t is 

    3/4 exp(-t) + 1/4

    and likewise for the other three base inputs.

  """
  if type(gen_time) !=  float :
    raise Exception("Time must be a float")
  for b in genome:
    if not ( b=='a' or b=='c' or b=='g' or b=='t' ):
       raise Exception("Invalid input genome")
  persistence_prob=(3.*M.exp(-gen_time)+1.)/4.
  new_genome=[]
  #print("Genome=",genome)
  for base in genome:
    p=random.random()
    if p < persistence_prob:
      new_base=base
    else: 
      if base=='a':
        seq=('c','g','t')
      if base=='c':
        seq=('a','g','t')
      if base=='g':
        seq=('a','c','t')
      if base=='t':
        seq=('a','c','g')
      new_base=random.choice(seq)
    new_genome.append(new_base)
  return(new_genome)

In [22]:
import copy

# The following routine, which applies mutations randomly, chooses the Jukes-Cantor algorithm,
# but some other model can be used by changing the function below.

def evolve(genome,time):
   return(jukesCantor(genome,time))

def generateHelper(initialGenome,listIn):
 time=listIn[0]
 newGenome=evolve(initialGenome,time)
 listIn[0]=newGenome
 if not ( len(listIn) == 1 or len(listIn) == 3):
    raise Exception("Lists must have one or three elements") 
 if len(listIn) == 1:
   return
 else:
   generateHelper(newGenome,listIn[1])
   generateHelper(newGenome,listIn[2])
   return 

def generateDriver(initialGenome,listIn):
  """
  This algorithm assumes a tree with input of the form:

  [ genTime, leftBranch, rightBranch]

  where the branches can either be tripleton lists of the format above
  or leaves represtented as a single nonnegative number enclosed in a list.
  genTime above is a non-negative real number. A deep copy of the input list is made,
  and the genetic times are replaced with the stochastically evolved genomes.

  An initial genone at the root of the binary tree is required as input, and this
  must be in a format of a list of base charcters, i.e., 'a', 'c', 'g', or 't'.

  """
  listInBis=copy.deepcopy(listIn)
  generateHelper(initialGenome,listInBis)
  return(listInBis)
   
initialGenome=10*['a']
#oldTree=[ 1.0, [1.0]  , [1.0] ]
oldTree=[ 1.0, [1.0, [1.0,[1.0],[1.0]]  , [1.0, [1.0], [1.0] ]], [2.0]  ]


newTree=generateDriver(initialGenome,oldTree)
print("Old Tree")
print(oldTree)
print("New Tree")
print(newTree)


Old Tree
[1.0, [1.0, [1.0, [1.0], [1.0]], [1.0, [1.0], [1.0]]], [2.0]]
New Tree
[['a', 'a', 'c', 't', 't', 'g', 'a', 't', 'c', 'a'], [['g', 'a', 'c', 'g', 'g', 't', 'c', 't', 'c', 't'], [['a', 't', 'c', 'g', 'g', 't', 'g', 't', 't', 't'], [['c', 't', 'c', 'a', 'g', 't', 'g', 'c', 'a', 't']], [['a', 't', 'c', 'a', 't', 't', 't', 't', 'c', 'g']]], [['c', 't', 'c', 'g', 'g', 'g', 'g', 't', 'a', 'a'], [['t', 't', 't', 'g', 'c', 'c', 'a', 't', 't', 'g']], [['c', 'c', 'c', 't', 'g', 't', 't', 't', 'a', 'a']]]], [['c', 'c', 'c', 'g', 't', 'g', 'a', 'a', 'a', 'c']]]


### Test of our program using Fitch algorithm and Sankoff algorithm for the data generated above

In [23]:
#genome sequences for the 5 species
sp1=newTree[1][1][1][0]
sp2=newTree[1][1][2][0]
sp3=newTree[1][2][1][0]
sp4=newTree[1][2][2][0]
sp5=newTree[2][0]

In [24]:
#create the data set
data_jc=[sp1,sp2,sp3,sp4,sp5]

In [25]:
#generate all possible tree for 5 species
t=labeled([1,2,3,4,5])

Let's try to find the most parsimonious for the data usign Sankoff algorithm. Then we use the cost matrix below

In [26]:
#cost matrix
c=[[0,1,1,1],[1,0,1,1],[1,1,0,1],[1,1,1,0]]
#set of character state
s_jc=['a','c','g','t']

In [27]:
#compute the most parsimonious tree using Sankoff algorithm
parsimonious_Sank(t,data_jc,s_jc,c)

([[[[1, 4], [3, 5]], 2],
  [[1, 4], [2, [3, 5]]],
  [[1, [2, [3, 5]]], 4],
  [1, [[2, [3, 5]], 4]],
  [[[1, 4], 2], [3, 5]],
  [[[[1, 4], 2], 3], 5],
  [[[[1, 4], 2], 5], 3]],
 20.0)

Infering phylogeny: Tree I Figure 21.1 (p. 366)

In [28]:
mouse='ACCAAAAAAACATCCAAACACCAACCCCAGCCCTTACGCAATAGCCATACAAAGAATATTATACTACTAAAAACTCAAATTAACTCTTTAATCTTTATACAACATTCCACCAACCTATCCACACAAAAAAACTCATATTTATCTAAATACGAACTTCACACAACCTTAACACATAAACATACCCCAGCCCAACACCCTTCCACAAATCCTTAATATACGCACCATAAATAAC'
m=[i for i in mouse]

In [29]:
bovine='ACCAAACCTGTCCCCACCATCTAACACCAACCCACATATACAAGCTAAACCAAAAATACCATACAACCATAAATAAGACTAATCTATTAAAATAACCCATTACGATACAAAATCCCTTTCGTCTAGATACAAACCACAACACACAATTAATACACACCACAATTACAATACTAAACTCCCATCCCACCAAATCACCCTCCATCAAATCCACAAATTACACAACCATTAACCC'
b=[i for i in bovine]

In [30]:
gibbon='ACTATACCCACCCAACTCGACCTACACCAATCCCCACATAGCACACAGACCAACAACCTCCCACCTTCCATACCAAGCCCCGACTTTACCGCCAACGCACCTCATCAAAACATACCTACAACACAAACAAATGCCCCCCCACCCTCCTTCTTCAAGCCCACTAGACCATCCTACCTTCCTAGCACGCCAAGCTCTCTACCATCAAACGCACAACTTACACATACAGAACCAC'
g=[i for i in gibbon]

In [31]:
orang='ACCCCACCCGTCTACACCAGCCAACACCAACCCCCACCTACTATACCAACCAATAACCTCTCAACCCCTAAACCAAACACTATCCCCAAAACCAACACACTCTACCAAAATACACCCCCAATTCACATCCGCACACCCCCACCCCCCCTGCCCACGTCCATCCCATCACCCTCTCCTCCCAACACCCTAAGCCACCTTCCTCAAAATCCAAAACCCACACAACCGAAACAAC'
o=[i for i in orang]

In [32]:
gorilla='ACCCCATTTATCCATAAAAACCAACACCAACCCCCATCTAACACACAAACTAATGACCCCCCACCCTCAAAGCCAAACACCAACCCTATAATCAATACGCCTTATCAAAACACACCCCCAACATAAACCCACGCACCCCCACCCCTTCCGCCCATGCTCACCACATCATCTCTCCCCTTCAACACCTCAATCCACCTCCCCCCAAATACACAATTCACACAAACAATACCAC'
go=[i for i in gorilla]

In [33]:
chimp='ACCCCATCCACCCATACAAACCAACATTACCCTCCATCCAATATACAAACTAACAACCTCCCACTCTTCAGACCGAACACCAATCTCACAACCAACACGCCCCGTCAAAACACCCCTTCAGCACAAATTCATACACCCCTACCTTTCCTACCCACGTTCACCACATCATCCCCCCCTCTCAACATCTTGACTCGCCTCTCTCCAAACACACAATTCACGCAAACAACGCCAC'
ch=[i for i in chimp]

In [34]:
human='ACCCCACTCACCCATACAAACCAACACCACTCTCCACCTAATATACAAATTAATAACCTCCCACCTTCAGAACTGAACGCCAATCTCATAACCAACACACCCCATCAAAGCACCCCTCCAACACAAACCCGCACACCTCCACCCCCCTCGTCTACGCTTACCACGTCATCCCTCCCTCTCAACACCTTAACTCACCTTCTCCCAAACGCACAATTCGCACACACAACGCCAC'
h=[i for i in human]

In [35]:
dataset=[m,b,g,o,go,ch,h]

In [36]:
list_tree=labeled([1,2,3,4,5,6,7]) #m:1,b:2,g:3,o:4,go:5,ch:6,h:7

In [37]:
#find the most parsimonious tree
cost_matrix=[[0,1,1,1],[1,0,1,1],[1,1,0,1],[1,1,1,0]]
parsimonious_Sank(list_tree,dataset,s,cost_matrix)

([[[1, [3, [4, [5, [6, 7]]]]], 2],
  [1, [2, [3, [4, [5, [6, 7]]]]]],
  [[1, 2], [3, [4, [5, [6, 7]]]]],
  [[[1, 2], 3], [4, [5, [6, 7]]]],
  [[[[1, 2], 3], 4], [5, [6, 7]]],
  [[[[[1, 2], 3], 4], 5], [6, 7]],
  [[[[[[1, 2], 3], 4], 5], 6], 7],
  [[[[[[1, 2], 3], 4], 5], 7], 6],
  [[[[[1, 2], 3], 4], [6, 7]], 5],
  [[[[1, 2], 3], [5, [6, 7]]], 4],
  [[[1, 2], [4, [5, [6, 7]]]], 3]],
 372.0)